In [1]:
!pip install rectools
!pip install pandas
!pip install numba
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 828 kB/s 
     |████████████████████████████████| 60 kB 3.7 MB/s 
     |████████████████████████████████| 88 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 310 kB 59.3 MB/s 
     |████████████████████████████████| 13.4 MB 59.8 MB/s 
     |████████████████████████████████| 188 kB 80.0 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-linux_x86_64.whl size=3825479 sha256=f80023a136223d0fde97cda22a120035a8cb6cd75f9f9f9f003142c9bbea4f01
  Stored in directory: /root/.cache/pip/wheels/00/ac/67/6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=747383 sha256=ef42e7df1ed6e4b35bce1e06c2eec98c13ce57306124bb817c5d94e88be7c471
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021

In [2]:
import pandas as pd
import numpy as np
import os

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization


## KION DATA

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import requests
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [5]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:  57%|█████▋    | 45.1M/78.8M [00:00<00:00, 446MiB/s]

In [6]:
import zipfile as zf

files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [7]:
interactions = pd.read_csv('data_original/interactions.csv')
Columns.Datetime = 'last_watch_dt'

In [8]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

### **Interactions prepare**

In [9]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [10]:
min_date = interactions[Columns.Datetime].min(), 
max_date = interactions[Columns.Datetime].max()

In [11]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [12]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [13]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
0,176549,9506,2021-05-11,4250,72.0,3
1,699317,1659,2021-05-29,8317,100.0,3
2,656683,7107,2021-05-09,10,0.0,1
3,864613,7638,2021-07-05,14483,100.0,3
4,964868,9506,2021-04-30,6725,100.0,3


In [14]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train.drop(train.query("total_dur < 300").index, inplace=True)

# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

train: (4985269, 6)
test: (490982, 6)


## User prepare

In [15]:
users.fillna('Unknown', inplace=True)
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [16]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [17]:
train_user_features = user_features.loc[user_features['id'].isin(train[Columns.User])]

## Item prepare


In [18]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

Genre

In [19]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()


,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


Content

In [20]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


In [21]:
item_features = pd.concat((genre_feature, content_feature))
item_features.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [22]:
train_item_features = item_features.loc[item_features['id'].isin(train[Columns.Item])]

### Save

In [23]:
interactions.to_csv('prepared_interactions.csv', index=False)
items.to_csv('prepared_items.csv', index=False)
users.to_csv('prepared_users.csv', index=False)

kion dataset download: 100%|██████████| 78.8M/78.8M [00:20<00:00, 446MiB/s]

In [24]:
item_features.to_csv( 'prepared_featured_items.csv', index=False)
user_features.to_csv('prepared_featured_users.csv', index=False)

In [25]:
train.to_csv('prepared_interactions_train.csv', index=False)
test.to_csv('prepared_interactions_test.csv', index=False)

train_user_features.to_csv('prepared_featured_users_train.csv', index=False)
train_item_features.to_csv('prepared_featured_items_train.csv', index=False)

In [26]:
us_dict_map = dict(enumerate(train['user_id'].unique()))
itm_dict_map = dict(enumerate(train['item_id'].unique()))

In [27]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

## Models

In [28]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

import warnings
warnings.filterwarnings('ignore')

In [29]:
NO_COMP = 64
LEARNING_RATE = 0.1
RANDOM_STATE = 42
NUM_THREADS = 12

In [30]:
lfm = LightFMWrapperModel(
        model = LightFM(
            no_components=NO_COMP,
            learning_rate=LEARNING_RATE, 
            loss='warp',
            rho=0.9,
            epsilon=1e-5,
            user_alpha=0,
            item_alpha=0,
            random_state=RANDOM_STATE,
        ),
        epochs=1,
        num_threads=NUM_THREADS,
    )
lfm.fit(dataset)

# ANN

In [31]:
import nmslib

In [32]:
import time

In [33]:
user_embeddings, item_embeddings = lfm.get_vectors(dataset)

In [34]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [35]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

pre shape:  (14019, 66)


(14019, 67)

In [36]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(756562, 67)

In [37]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}


In [38]:
# Number of neighbors 
K=10
# Space name should correspond to the space name 
# used for brute-force search
space_name='negdotprod'

In [39]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

14019

In [40]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 0.914488


In [41]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [42]:
query_matrix = augmented_user_embeddings[:1000, :]

In [43]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=0.043361 (sec), per query=0.000043 (sec), per query adjusted for thread number=0.000173 (sec)


In [44]:
recs = {int(us_dict_map[idx_u]): [int(itm_dict_map[idx_i]) for idx_i in predict[0]] for idx_u, predict in enumerate(nbrs)}

In [49]:
import json

In [50]:
with open('ann_recs.json', 'w') as f:
    json.dump(recs, f)

# AVATARS

In [65]:
maximums = users.max()

In [66]:
maximums.user_id

1097560

Мама с ребенком (мелодрамы и мультики)

In [67]:
users = users.append({
    Columns.User: maximums.user_id+1,
    'age': 'age_25_34',
    'income': 'income_20_40',
    'sex': 'Ж',
    'kids_flg': 1
}, ignore_index = True
)

In [68]:
avatar1_titles = items[items['genres'].str.contains('мультфильмы') | items['genres'].str.contains('мелодрамы')].sample(5, random_state=RANDOM_STATE)['title'].values
avatar1_titles

array(['Аладдин', 'Это моя собака', 'Исцеление (2013)', 'Пять комнат',
       'Грузовичок Пик'], dtype=object)

In [79]:
avatar1_interactions = pd.DataFrame({
    "user_id": maximums.user_id+1, 
    "title": avatar1_titles,
    "last_watch_dt": np.full(5, fill_value='2020-06-17'),
    "total_dur": np.full(5, fill_value=np.nan),
    "watched_pct":[100, 100, 70, 70, 70],
    "weight": [5, 3, 3, 5, 1],
    })
avatar1_interactions = avatar1_interactions.merge(items[["item_id", "title", "genres"]], on="title")

In [80]:
avatar1_interactions

,user_id,title,last_watch_dt,total_dur,watched_pct,weight,item_id,genres
0,1097561,Аладдин,2020-06-17,NaN,100,5,12421,"мультфильм, приключения, мюзиклы, мелодрамы, ф..."
1,1097561,Аладдин,2020-06-17,NaN,100,5,11310,"фэнтези, приключения, мелодрамы"
2,1097561,Это моя собака,2020-06-17,NaN,100,3,9925,"русские, мелодрамы"
3,1097561,Исцеление (2013),2020-06-17,NaN,70,3,7040,"русские, мелодрамы"
4,1097561,Пять комнат,2020-06-17,NaN,70,5,3333,"мелодрамы, комедии"
5,1097561,Грузовичок Пик,2020-06-17,NaN,70,1,8453,"развлекательные, развитие, мультфильмы"


Пожелой мужчина, исторические фильмы

In [71]:
users = users.append({
    Columns.User: maximums.user_id+2,
    'age': 'age_65_inf',
    'income': 'income_40_60',
    'sex': 'М',
    'kids_flg': 0
}, ignore_index = True
)

In [72]:
avatar2_titles = items[items['genres'].str.contains('исторические')].sample(5, random_state=RANDOM_STATE)['title'].values
avatar2_titles

array(['Заговор против Америки', '1812', 'Тачанка с юга',
       'Пираты семи морей: Черная борода', 'Падение последней империи'],
      dtype=object)

In [73]:
avatar2_interactions = pd.DataFrame({
    "user_id": maximums.user_id+2, 
    "title": avatar2_titles,
    "last_watch_dt": np.full(5, fill_value='2020-06-17'),
    "total_dur": np.full(5, fill_value=np.nan),
    "watched_pct":[70, 80, 70, 70, 70],
    "weight": [5, 5, 5, 5, 1],
    })
avatar2_interactions = avatar2_interactions.merge(items[["item_id", "title", "genres"]], on="title")

In [74]:
avatar2_interactions

,user_id,title,last_watch_dt,total_dur,watched_pct,weight,item_id,genres
0,1097562,Заговор против Америки,2020-06-17,NaN,70,5,10620,"драмы, исторические, фантастика"
1,1097562,1812,2020-06-17,NaN,80,5,3726,"русские, познавательные, драмы, исторические, ..."
2,1097562,Тачанка с юга,2020-06-17,NaN,70,5,4243,"исторические, советские, приключения"
3,1097562,Пираты семи морей: Черная борода,2020-06-17,NaN,70,5,8102,"приключения, зарубежные, исторические, военные..."
4,1097562,Падение последней империи,2020-06-17,NaN,70,1,9431,"приключения, драмы, зарубежные, исторические, ..."


Неразделенная любовь (зарубежные драмы)

In [75]:
users = users.append({
    Columns.User: maximums.user_id+3,
    'age': 'age_18_24',
    'income': 'income_0_20',
    'sex': 'Ж',
    'kids_flg': 0
}, ignore_index = True
)

In [76]:
avatar3_titles = items[items['genres'].str.contains('драмы') & items['genres'].str.contains('зарубежные')].sample(5, random_state=RANDOM_STATE)['title'].values
avatar3_titles

array(['Стриптизерши', 'Мэгги и Бьянка в Академии моды',
       '[4K] На глубине 6 футов', 'До свидания там, наверху',
       'Сокровище нации'], dtype=object)

In [77]:
avatar3_interactions = pd.DataFrame({
    "user_id": maximums.user_id+3, 
    "title": avatar3_titles,
    "last_watch_dt": np.full(5, fill_value='2020-06-17'),
    "total_dur": np.full(5, fill_value=np.nan),
    "watched_pct":[100, 90, 90, 90, 90],
    "weight": [5, 4, 1, 3, 2]})
avatar3_interactions = avatar3_interactions.merge(items[["item_id", "title", "genres"]], on="title")

In [78]:
avatar3_interactions

,user_id,title,last_watch_dt,total_dur,watched_pct,weight,item_id,genres
0,1097563,Стриптизерши,2020-06-17,NaN,100,5,16487,"драмы, зарубежные, криминал, комедии"
1,1097563,Мэгги и Бьянка в Академии моды,2020-06-17,NaN,90,4,9166,"драмы, зарубежные, мелодрамы, семейное, комедии"
2,1097563,[4K] На глубине 6 футов,2020-06-17,NaN,90,1,407,"биография, приключения, драмы, зарубежные, три..."
3,1097563,"До свидания там, наверху",2020-06-17,NaN,90,3,3245,"драмы, зарубежные"
4,1097563,Сокровище нации,2020-06-17,NaN,90,2,11458,"боевики, триллеры, приключения"
5,1097563,Сокровище нации,2020-06-17,NaN,90,2,15647,"драмы, зарубежные"


In [86]:
avatar1_interactions[Columns.Datetime] = pd.to_datetime(avatar1_interactions[Columns.Datetime], format='%Y-%m-%d')
avatar2_interactions[Columns.Datetime] = pd.to_datetime(avatar2_interactions[Columns.Datetime], format='%Y-%m-%d')
avatar3_interactions[Columns.Datetime] = pd.to_datetime(avatar3_interactions[Columns.Datetime], format='%Y-%m-%d')

In [81]:
train

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
0,176549,9506,2021-05-11,4250,72.0,3
1,699317,1659,2021-05-29,8317,100.0,3
3,864613,7638,2021-07-05,14483,100.0,3
4,964868,9506,2021-04-30,6725,100.0,3
5,1032142,6686,2021-05-13,11286,100.0,3
...,...,...,...,...,...,...
5476242,268216,3071,2021-04-21,5752,98.0,3
5476244,438585,7829,2021-08-02,6804,100.0,3
5476245,786732,4880,2021-05-12,753,0.0,1
5476247,546862,9673,2021-04-13,2308,49.0,3


In [87]:
train_avatars = pd.concat([
    train, 
    avatar1_interactions.drop(['title', 'genres'], axis=1),
    avatar2_interactions.drop(['title', 'genres'], axis=1), 
    avatar3_interactions.drop(['title', 'genres'], axis=1),
], sort=False)
train_avatars.tail(30)

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
5476231,503488,15997,2021-04-01,397.0,7.0,1
5476232,699405,4702,2021-05-20,1709.0,28.0,3
5476233,399997,14901,2021-07-31,489.0,8.0,1
5476234,814537,1856,2021-05-15,713.0,79.0,3
5476235,977542,13126,2021-07-04,1830.0,26.0,3
5476239,610017,7107,2021-05-10,1133.0,75.0,3
5476240,802291,512,2021-08-08,6990.0,100.0,3
5476241,1073802,9927,2021-08-07,6425.0,97.0,3
5476242,268216,3071,2021-04-21,5752.0,98.0,3
5476244,438585,7829,2021-08-02,6804.0,100.0,3


In [98]:
dataset_av = Dataset.construct(
    interactions_df=train_avatars,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "countries"],
)

In [99]:
NO_COMP = 64
LEARNING_RATE = 0.1
RANDOM_STATE = 42
NUM_THREADS = 12
K_RECOS = 10

In [100]:
lfm = LightFMWrapperModel(
        model = LightFM(
            no_components=NO_COMP,
            learning_rate=LEARNING_RATE, 
            loss='warp',
            rho=0.9,
            epsilon=1e-5,
            user_alpha=0,
            item_alpha=0,
            random_state=RANDOM_STATE,
        ),
        epochs=1,
        num_threads=NUM_THREADS,
    )
lfm.fit(dataset_av)

In [112]:
av = pd.concat((avatar1_interactions, avatar2_interactions, avatar3_interactions))

In [113]:
av_ids = av['user_id'].unique()

In [114]:
recs_av = lfm.recommend(
    users=av_ids,
    dataset=dataset_av,
    k=K_RECOS,
    filter_viewed=True,
)

In [116]:
recoms = recs_av.merge( items[['item_id', 'title', 'countries', 'genre']], on='item_id')

In [117]:
recoms

,user_id,item_id,score,rank,title,countries,genre
0,1097561,8131,19.808957,1,"Чтобы увидеть радугу, нужно пережить дождь",Украина,"[зарубежные, криминал, мелодрамы]"
1,1097561,13191,17.220926,2,Ночь в музее,"США, Великобритания","[семейное, фэнтези, приключения, комедии]"
2,1097561,9234,14.787405,3,Бои без правил,"Великобритания, Франция, Китай, Камбоджа, США,...","[биография, криминал, драмы, спорт, триллеры]"
3,1097561,4398,14.041927,4,Муж на час,Украина,"[мелодрамы, комедии]"
4,1097561,8083,13.123398,5,Спящая красавица,США,"[мюзиклы, мультфильм, фэнтези, мелодрамы]"
5,1097561,3606,11.779671,6,Грешники и святые,США,"[боевики, триллеры, криминал]"
6,1097561,12988,10.760368,7,Гномео и Джульетта,"Великобритания, США, Канада","[мелодрамы, мультфильм, приключения, комедии]"
7,1097561,10323,10.680384,8,Университет монстров,США,"[мультфильм, фэнтези, приключения, комедии]"
8,1097561,13069,10.470164,9,Книга жизни,США,"[мультфильм, приключения, мюзиклы, мелодрамы, ..."
9,1097561,7571,10.426362,10,100% волк,"Австралия, Бельгия","[мультфильм, приключения, семейное, фэнтези, к..."


**Рекомендации для мамы с ребенком (мультфильмы + мелодрамы)**

In [120]:
recoms[recoms.user_id == 1097561]

,user_id,item_id,score,rank,title,countries,genre
0,1097561,8131,19.808957,1,"Чтобы увидеть радугу, нужно пережить дождь",Украина,"[зарубежные, криминал, мелодрамы]"
1,1097561,13191,17.220926,2,Ночь в музее,"США, Великобритания","[семейное, фэнтези, приключения, комедии]"
2,1097561,9234,14.787405,3,Бои без правил,"Великобритания, Франция, Китай, Камбоджа, США,...","[биография, криминал, драмы, спорт, триллеры]"
3,1097561,4398,14.041927,4,Муж на час,Украина,"[мелодрамы, комедии]"
4,1097561,8083,13.123398,5,Спящая красавица,США,"[мюзиклы, мультфильм, фэнтези, мелодрамы]"
5,1097561,3606,11.779671,6,Грешники и святые,США,"[боевики, триллеры, криминал]"
6,1097561,12988,10.760368,7,Гномео и Джульетта,"Великобритания, США, Канада","[мелодрамы, мультфильм, приключения, комедии]"
7,1097561,10323,10.680384,8,Университет монстров,США,"[мультфильм, фэнтези, приключения, комедии]"
8,1097561,13069,10.470164,9,Книга жизни,США,"[мультфильм, приключения, мюзиклы, мелодрамы, ..."
9,1097561,7571,10.426362,10,100% волк,"Австралия, Бельгия","[мультфильм, приключения, семейное, фэнтези, к..."


Как видно, мультфильмы и мелодрамы преобладают, но есть и неподходящие категории

**Рекомендации для люителя исторических фильмов**

In [121]:
recoms[recoms.user_id == 1097562]

,user_id,item_id,score,rank,title,countries,genre
10,1097562,10794,22.110605,1,Вы заказывали убийство,Россия,"[русские, детективы]"
11,1097562,4389,17.964733,2,Приходи свободным,СССР,"[драмы, исторические, советские, военные]"
12,1097562,4748,16.162696,3,Отключение,Бельгия,"[драмы, зарубежные, триллеры, детективы]"
13,1097562,160,15.566936,4,Крейсер «Варяг»,СССР,"[драмы, исторические, советские, военные]"
14,1097562,3375,15.461874,5,Провал операции «Большая Медведица»,СССР,"[драмы, советские, военные]"
15,1097562,15279,15.410039,6,Горький можжевельник,СССР,"[драмы, русские, военные]"
16,1097562,5079,14.869333,7,Красавица и воры,Россия,"[русские, детективы, мелодрамы]"
17,1097562,3248,14.812299,8,Хорнблауэр,Великобритания,"[драмы, зарубежные, военные, приключения]"
18,1097562,5172,14.802380,9,Чистая проба,Россия,"[детективы, русские, приключения]"
19,1097562,1885,14.715634,10,Личные мотивы,Россия,"[медицинские, русские, детективы]"


**Рекомендации для пользователя с зарубежными драмами**

In [122]:
recoms[recoms.user_id == 1097563]

,user_id,item_id,score,rank,title,countries,genre
20,1097563,13085,28.213650,1,Буу!,США,"[зарубежные, мистика, ужасы]"
21,1097563,16178,27.874390,2,Идеалист,"Дания, США","[зарубежные, триллеры]"
22,1097563,11964,27.011906,3,Изменение духа,США,"[боевики, зарубежные, фантастика]"
23,1097563,5836,26.818923,4,Хозяин джунглей,"Аргентина, Мексика","[драмы, зарубежные, вестерн]"
24,1097563,10433,26.106274,5,Цирк мертвецов,США,"[боевики, зарубежные, триллеры, ужасы]"
25,1097563,15322,25.533422,6,Самая красивая (на казахском языке),Казахстан,"[зарубежные, криминал, комедии]"
26,1097563,13539,25.526441,7,В тихом омуте,Франция,"[зарубежные, комедии]"
27,1097563,5654,25.489154,8,Случайно беременна,Франция,"[зарубежные, комедии]"
28,1097563,10006,25.460706,9,Не стучи дважды,Великобритания,"[зарубежные, ужасы]"
29,1097563,1611,24.931675,10,Мой дядя Джон – зомби,США,"[зарубежные, комедии, ужасы]"
